# Replication - Habitat 2.3. Testing for why these numbers are much higher than my thesis results

### Import libraries and set up environment

In [1]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [2]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [3]:
# Intersect the dissolved 30 km MPA layer with the habitats

in_feature_1 = "WDPA_30km_dis"
in_feature_2 = "habitats_dis_merge"
out_feature = "\\MPA_30km_habitat_int_Replic2_3"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseIntersect(
    in_features=[in_feature_1,in_feature_2],
    out_feature_class=output_gdb + out_feature,
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"

)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MPA_30km_habitat_int_Replic2_3'>

In [4]:
# trying this step without the singlepart step

# Repair geoemtry on intersect and calculate area of intersecting bits

in_feature = "MPA_30km_habitat_int_Replic2_3_TEST"
# out_feature = "MPA_30km_habitat_int_singl_Replic2_3"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def repair_geo_area():
    #repair geometry
    arcpy.management.RepairGeometry(
        in_features= in_feature,
        delete_null="DELETE_NULL",
        validation_method="OGC"
    )
    
#     #make the intersected layer signlepart to ensure all polygons are individual
#     arcpy.management.MultipartToSinglepart(
#         in_features= in_feature,
#         out_feature_class= out_feature
#     )
            
    # insert "area" field (double) and calculate area in km2 in Mollweide
    arcpy.management.CalculateGeometryAttributes(
        in_features=in_feature,
        geometry_property="area AREA",
        length_unit="",
        area_unit="SQUARE_KILOMETERS",
        coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
        coordinate_format="SAME_AS_INPUT"
    )

repair_geo_area()


In [5]:
# Only keep features with an area greater than .24 km2

# Select features with an area greater than .24 km2 and make a new layer from that and export it to excel

# Set a folder to store the output tables
table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Replication"

in_feature = "MPA_30km_habitat_int_Replic2_3_TEST"
out_feature = "MPA_30km_habitat_int_big_Replic2_3_TEST"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def select_big_excel():
    # make a layer of polygons with an area greater than .24 km2    
    arcpy.conversion.ExportFeatures(
        in_features= in_feature,
        out_features= out_feature,
        where_clause="area >= 0.24",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=None,
        sort_field=None
    )
    
    # Export table to excel
    # Not sure why I had to use the following two lines instead of just using the TableToExcel expression, but I did
    excel_filename = out_feature + ".xlsx"
    excel_filepath = os.path.join(table_outputs_folder, excel_filename)  # Build the full file path

    arcpy.conversion.TableToExcel(
        Input_Table= in_feature,
        Output_Excel_File= excel_filepath,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )
    
select_big_excel()

In [6]:
# Convert the output features to a Pandas DataFrame
df = pd.DataFrame.spatial.from_featureclass("MPA_30km_habitat_int_big_Replic2_3_TEST")


In [7]:
# Display the DataFrame
df.head()

,OBJECTID,FID_WDPA_30km_dis,FID_habitats_dis_merge,Id,gridcode,Habitat,area,SHAPE
0,1,944,2416,16708.0,1.0,shelf_soft,0.791052,"{""rings"": [[[-6463622.1938000005, 2273129.8806..."
1,2,1268,2428,16720.0,1.0,shelf_soft,2.502463,"{""rings"": [[[-6453094.283399999, 2272195.40169..."
2,3,4699,2669,16997.0,1.0,shelf_soft,0.873251,"{""rings"": [[[-6282481.965500001, 2186223.34500..."
3,4,4699,2671,16999.0,1.0,shelf_soft,0.873251,"{""rings"": [[[-6285285.4022, 2185288.8662], [-6..."
4,5,4699,2673,17001.0,1.0,shelf_soft,0.873251,"{""rings"": [[[-6286219.881100001, 2184354.38729..."


In [8]:
# Make pivot tables
pivot_MPA_habitat = df.pivot_table(
     index = "Habitat",
     values = "OBJECTID",
     aggfunc = "count"
).reset_index()
pivot_MPA_habitat.columns = ["Habitat", "count"]
print(pivot_MPA_habitat)



           Habitat  count
0            coral   1567
1        deep_hard     30
2        deep_soft     33
3         mangrove   2720
4        saltmarsh   1280
5         seagrass    794
6   seamount_knoll    126
7     shallow_hard    360
8     shallow_soft    862
9       shelf_hard    399
10      shelf_soft    515
11      slope_hard     65
12      slope_soft    128


In [12]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Replication"
table = pivot_MPA_habitat
output_name = "MPA_30km_habitat_int_big_Replic2_3_TEST_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))

Reminder to add in the marine ecoregions/pelagic provinces that fall within the study area but don't have any intersections with MPAs so they dont show up in the results. I think there are 3 or so of them.